### Process all of the .csv files provided, combining all of the individual datasets into a single dataset for analysis

clearly demonstrate how you have chosen to deal with any data wrangling issues (e.g. missing data, unrecognised activity codes, incompatible date/time formats).

Ensure that different date/time formats are resolved, returning error messages or handling exceptions if incompatible .csv files are provided)

In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
def read_and_merge_csvs(folder_name):
    from os import listdir
    from os.path import isfile, join

    try:
        csv_filenames = [f for f in listdir(folder_name) if '.csv' in f]
    
    except Exception as e:
        print("Error during reading csvs:", e)
    
    try:
        df_lst = []
        for i, f in enumerate(csv_filenames):
            df = pd.read_csv(join(folder_name, f))
            df.insert(0, "CSVNumber", i, True)
            df_lst.append(df)
        
        df = pd.concat(df_lst)
            
#         df = pd.concat([pd.read_csv(join(folder_name, f)) for f in csv_filenames])
        
    except Exception as e:
        print("Error during processing csvs:", e)
        
    return df

In [3]:
df = read_and_merge_csvs('data')

In [4]:
df = df.loc[:, :"EnjoymentScore"] # removing further columns

In [5]:
df

,CSVNumber,PrimaryActivityCode,SecondaryActivityCode,DescriptiveLabel,StartDate,UniBathWeekNo,DayOfWeek,StartTime,EndTime,DurationMins,EnjoymentScore
0,0,S801,,Sleeping,08/03/2021,24.0,Monday,00:00,07:00,420.0,2.0
1,0,O733,,Getting ready,08/03/2021,24.0,Monday,07:00,07:30,29.0,1.0
2,0,X893,UD415,Exercising while listening music,08/03/2021,24.0,Monday,07:30,08:15,45.0,2.0
3,0,ED152,,Breakfast,08/03/2021,24.0,Monday,08:15,08:45,29.0,1.0
4,0,CW982,,ADS Lab 3,08/03/2021,24.0,Monday,08:45,11:15,150.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
30,79,X893,NaN,Walk,12/03/2021,24.0,Friday,16:00,17:00,60.0,2.0
31,79,EL642,UD415,NaN,12/03/2021,24.0,Friday,17:00,18:30,90.0,-1.0
32,79,O733,NaN,Dinner,12/03/2021,24.0,Friday,18:30,19:00,30.0,2.0
33,79,EL642,UD415,NaN,12/03/2021,24.0,Friday,19:00,21:30,150.0,-1.0


#### finding missing/unrecognised data in the wrangled dataset

In [6]:
# Check missing/unrecognised Primary/SecondaryActivityCode
# capitalise and remove spaces in letters of activity codes
df["PrimaryActivityCode"] = df["PrimaryActivityCode"].str.upper()
df["PrimaryActivityCode"] = df["PrimaryActivityCode"].str.strip()
df["SecondaryActivityCode"] = df["SecondaryActivityCode"].str.upper()
df["SecondaryActivityCode"] = df["SecondaryActivityCode"].str.strip()

print("Converted Primary and Secondary activity code strings into upper letter")

# drop rows that has NaN primary codes
df = df.drop(pd.isnull(df["PrimaryActivityCode"]))

print("Dropped nan primary codes")

# check if primary code is missing
# Change secondary code to nan if Primary code == secondary code in same row
df['SecondaryActivityCode'].loc[df['PrimaryActivityCode'] == df['SecondaryActivityCode']] = np.nan

print("Converted secondary activity codes to NaN where it is the same to primary")

# change empty string ' ' in secondary code to np.nan
index = np.where(df['SecondaryActivityCode'].apply(lambda x: (x == ' ') or (x == '')))
df['SecondaryActivityCode'].iloc[index] = np.nan

print("Converted empty strings to NaN")


Converted Primary and Secondary activity code strings into upper letter
Dropped nan primary codes
Converted secondary activity codes to NaN where it is the same to primary
Converted empty strings to NaN


C:\Users\jihoon-thinkpad\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-6-6a7dfe5307e4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SecondaryActivityCode'].loc[df['PrimaryActivityCode'] == df['SecondaryActivityCode']] = np.nan
<ipython-input-6-6a7dfe5307e4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sec

In [7]:
# verify
print(np.where(df['SecondaryActivityCode'].apply(lambda x: x == ' ')))
print(df[df['PrimaryActivityCode'] == df['SecondaryActivityCode']])

(array([], dtype=int64),)
Empty DataFrame
Columns: [CSVNumber, PrimaryActivityCode, SecondaryActivityCode, DescriptiveLabel, StartDate, UniBathWeekNo, DayOfWeek, StartTime, EndTime, DurationMins, EnjoymentScore]
Index: []


In [8]:
# check if any invalid codes exist (not in the code list)
lst_ActivityCode = ['ED145', 'EL642', 'X893', 'H179', 'L418', 'R523', 'S801',
                          'T695', 'UD415', 'PW101', 'CW982', 'O733']

lst_invalid_PrimaryActivityCode = np.unique([i for i in df["PrimaryActivityCode"] if (i not in lst_ActivityCode)])
# don't drop rows with nan values in secondaryactivitycode using i==i (return False if nan)
lst_invalid_SecondaryActivityCode = np.unique([i for i in df["SecondaryActivityCode"] if (i not in lst_ActivityCode) and (i==i)])


print("invalid PrimaryActivityCode:\n", lst_invalid_PrimaryActivityCode)
print("invalid SecondaryActivityCode:\n", lst_invalid_SecondaryActivityCode)

# drop rows that contain invalid codes
primary_row_labels = [(i in lst_invalid_PrimaryActivityCode) or (i!=i) for i in df["PrimaryActivityCode"]]
secondary_row_labels = [i in lst_invalid_SecondaryActivityCode for i in df["SecondaryActivityCode"]]
row_labels = np.logical_or(primary_row_labels, secondary_row_labels)
row_labels = [not i for i in row_labels]  # valid list
df = df.iloc[row_labels]

print("Dropped rows with invalid activity codes")


invalid PrimaryActivityCode:
 ['CW892' 'DI297' 'ED152' 'EL162' 'LE452' 'R253' 'SHOPPING' 'UB415' 'nan']
invalid SecondaryActivityCode:
 ['CHR' 'ED152' 'ENT' 'ESS' 'OTH' 'PRO' 'SA114' 'SA151']
Dropped rows with invalid activity codes


<ipython-input-8-14c52813cbd5>:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  primary_row_labels = [(i in lst_invalid_PrimaryActivityCode) or (i!=i) for i in df["PrimaryActivityCode"]]
<ipython-input-8-14c52813cbd5>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  secondary_row_labels = [i in lst_invalid_SecondaryActivityCode for i in df["SecondaryActivityCode"]]


In [9]:
# verify if I haven't deleted rows with NaN in secondary activity code
print("NaN in Primary:", any(pd.isnull(df["PrimaryActivityCode"])))
print("NaN in Secondary:", any(pd.isnull(df["SecondaryActivityCode"])))


lst_invalid_PrimaryActivityCode = np.unique([i for i in df["PrimaryActivityCode"] if (i not in lst_ActivityCode)])
# don't drop rows with nan values in secondaryactivitycode using i==i (return False if nan)
lst_invalid_SecondaryActivityCode = np.unique([i for i in df["SecondaryActivityCode"] if (i not in lst_ActivityCode) and (i==i)])


print("invalid PrimaryActivityCode:", lst_invalid_PrimaryActivityCode)
print("invalid SecondaryActivityCode:", lst_invalid_SecondaryActivityCode)



NaN in Primary: False
NaN in Secondary: True
invalid PrimaryActivityCode: []
invalid SecondaryActivityCode: []


In [10]:
df.reset_index(drop=True)

,CSVNumber,PrimaryActivityCode,SecondaryActivityCode,DescriptiveLabel,StartDate,UniBathWeekNo,DayOfWeek,StartTime,EndTime,DurationMins,EnjoymentScore
0,0,X893,UD415,Exercising while listening music,08/03/2021,24.0,Monday,07:30,08:15,45.0,2.0
1,0,CW982,NaN,ADS Lab 3,08/03/2021,24.0,Monday,08:45,11:15,150.0,1.0
2,0,H179,NaN,Room and desk cleaning,08/03/2021,24.0,Monday,11:15,11:45,29.0,-1.0
3,0,CW982,NaN,Preparing for ENTP presentation,08/03/2021,24.0,Monday,11:45,12:45,60.0,1.0
4,0,EL642,UD415,Online ENTP lecture,08/03/2021,24.0,Monday,13:15,14:15,59.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2700,79,X893,NaN,Walk,12/03/2021,24.0,Friday,16:00,17:00,60.0,2.0
2701,79,EL642,UD415,NaN,12/03/2021,24.0,Friday,17:00,18:30,90.0,-1.0
2702,79,O733,NaN,Dinner,12/03/2021,24.0,Friday,18:30,19:00,30.0,2.0
2703,79,EL642,UD415,NaN,12/03/2021,24.0,Friday,19:00,21:30,150.0,-1.0


In [11]:
# Check StartDate
# Check the date format is correct
# Check the date is in correct time frame (Feb~March 2021)
print('Added StartDateTime and EndDateTime columns that merge date and time values')

df['StartDate'] = pd.to_datetime(df['StartDate'], errors='coerce')
print("Converted StartTime string to datetime format")

# drop rows where date is nan
df = df.loc[df['StartDate'] == df['StartDate']]
# df['StartDate'] = pd.to_datetime(df['StartDate'], errors='coerce')
print("Dropped rows with NaN dates")

Added StartDateTime and EndDateTime columns that merge date and time values
Converted StartTime string to datetime format
Dropped rows with NaN dates


In [12]:
df['StartDate'].unique()

array(['2021-08-03T00:00:00.000000000', '2021-09-03T00:00:00.000000000',
       '2021-10-03T00:00:00.000000000', '2021-05-03T00:00:00.000000000',
       '2021-06-03T00:00:00.000000000', '2021-07-03T00:00:00.000000000',
       '2021-03-08T00:00:00.000000000', '2021-03-09T00:00:00.000000000',
       '2021-03-10T00:00:00.000000000', '2021-02-24T00:00:00.000000000',
       '2021-02-25T00:00:00.000000000', '2021-02-26T00:00:00.000000000',
       '2021-03-04T00:00:00.000000000', '2021-03-05T00:00:00.000000000',
       '2021-03-06T00:00:00.000000000', '2021-03-03T00:00:00.000000000',
       '2021-04-03T00:00:00.000000000', '2021-03-07T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-02-03T00:00:00.000000000',
       '2021-03-11T00:00:00.000000000', '2021-03-02T00:00:00.000000000',
       '2020-03-07T00:00:00.000000000', '2020-03-08T00:00:00.000000000',
       '2020-03-09T00:00:00.000000000', '2021-03-01T00:00:00.000000000',
       '2021-02-04T00:00:00.000000000', '2021-11-03

There are some StartDate values with swapped month and date. It is likely that 2021-06-03 is actually 2021-03-06. This issue needs to be addressed as dropping these rows will reduce the amount of data significantly.

In [13]:
def month_date_swapper(StartDate):
    
    import datetime as dt
    
    months = np.array(StartDate.dt.month)
    dates = np.array(StartDate.dt.day)
    
    for i, date in enumerate(StartDate):
#         print(date, months[i], dates[i], ((months[i] != 2) and (months[i] != 3)), ((dates[i] == 2) or (dates[i] == 3)))
        if ((months[i] != 2) and (months[i] != 3)) and ((dates[i] == 2) or (dates[i] == 3)):
            StartDate.iloc[i] = dt.datetime.strftime(date, "%Y-%d-%m %H:%M:%S")
    
    StartDate = pd.to_datetime(StartDate, errors='coerce')        
    return StartDate

In [14]:
df['StartDate'] = month_date_swapper(df['StartDate'])

C:\Users\jihoon-thinkpad\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [15]:
df['StartDate'].unique()

array(['2021-03-08T00:00:00.000000000', '2021-03-09T00:00:00.000000000',
       '2021-03-10T00:00:00.000000000', '2021-03-05T00:00:00.000000000',
       '2021-03-06T00:00:00.000000000', '2021-03-07T00:00:00.000000000',
       '2021-02-24T00:00:00.000000000', '2021-02-25T00:00:00.000000000',
       '2021-02-26T00:00:00.000000000', '2021-03-04T00:00:00.000000000',
       '2021-03-03T00:00:00.000000000', '2021-03-01T00:00:00.000000000',
       '2021-02-03T00:00:00.000000000', '2021-03-11T00:00:00.000000000',
       '2021-03-02T00:00:00.000000000', '2020-03-07T00:00:00.000000000',
       '2020-03-08T00:00:00.000000000', '2020-03-09T00:00:00.000000000',
       '2021-02-04T00:00:00.000000000', '2021-03-12T00:00:00.000000000',
       '2020-03-01T00:00:00.000000000', '2021-02-23T00:00:00.000000000',
       '2021-02-02T00:00:00.000000000', '2019-02-08T00:00:00.000000000',
       '2021-02-27T00:00:00.000000000', '2020-03-10T00:00:00.000000000',
       '2020-03-11T00:00:00.000000000', '2021-02-28

In [16]:
mask = ((df['StartDate'] >= '2021-02-19') & (df['StartDate'] <= '2021-03-12')) # date that this coursework was set and due
df = df.loc[mask]
print("Dropped rows where date is outside the range of coursework dates")

Dropped rows where date is outside the range of coursework dates


In [17]:
df.reset_index(drop=True)

,CSVNumber,PrimaryActivityCode,SecondaryActivityCode,DescriptiveLabel,StartDate,UniBathWeekNo,DayOfWeek,StartTime,EndTime,DurationMins,EnjoymentScore
0,0,X893,UD415,Exercising while listening music,2021-03-08,24.0,Monday,07:30,08:15,45.0,2.0
1,0,CW982,NaN,ADS Lab 3,2021-03-08,24.0,Monday,08:45,11:15,150.0,1.0
2,0,H179,NaN,Room and desk cleaning,2021-03-08,24.0,Monday,11:15,11:45,29.0,-1.0
3,0,CW982,NaN,Preparing for ENTP presentation,2021-03-08,24.0,Monday,11:45,12:45,60.0,1.0
4,0,EL642,UD415,Online ENTP lecture,2021-03-08,24.0,Monday,13:15,14:15,59.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2379,79,X893,NaN,Walk,2021-03-12,24.0,Friday,16:00,17:00,60.0,2.0
2380,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,17:00,18:30,90.0,-1.0
2381,79,O733,NaN,Dinner,2021-03-12,24.0,Friday,18:30,19:00,30.0,2.0
2382,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,19:00,21:30,150.0,-1.0


In [18]:
# check DayOfWeek
print(df['DayOfWeek'].unique())

['Monday' 'Tuesday' 'Wednesday' 'Friday' 'Saturday' 'Sunday' 'Thursday'
 nan 'Sat' 'Fri' 'Thu' 'Wendnesday' 'Wed' 'Thur']


In [19]:
days_list = ['Monday', 'Tuesday', 'Wednesday', 'Friday', 'Saturday', 'Sunday', 'Thursday']
mask = (df['DayOfWeek'].isin(days_list))

df = df[mask]

In [20]:
# check DayOfWeek after mask
print(df['DayOfWeek'].unique())

['Monday' 'Tuesday' 'Wednesday' 'Friday' 'Saturday' 'Sunday' 'Thursday']


In [21]:
# Check UniBathWeekNo
# Check the week range is between 20~25 and integer
df['UniBathWeekNo'].unique()

array([24., 23., 22., 25., nan, 10.])

In [22]:
# drop NaN and values not between 20-25
mask = (df['UniBathWeekNo'] >= 20) & (df['UniBathWeekNo'] <= 25)
df = df[mask]

In [23]:
df.reset_index(drop=True)

,CSVNumber,PrimaryActivityCode,SecondaryActivityCode,DescriptiveLabel,StartDate,UniBathWeekNo,DayOfWeek,StartTime,EndTime,DurationMins,EnjoymentScore
0,0,X893,UD415,Exercising while listening music,2021-03-08,24.0,Monday,07:30,08:15,45.0,2.0
1,0,CW982,NaN,ADS Lab 3,2021-03-08,24.0,Monday,08:45,11:15,150.0,1.0
2,0,H179,NaN,Room and desk cleaning,2021-03-08,24.0,Monday,11:15,11:45,29.0,-1.0
3,0,CW982,NaN,Preparing for ENTP presentation,2021-03-08,24.0,Monday,11:45,12:45,60.0,1.0
4,0,EL642,UD415,Online ENTP lecture,2021-03-08,24.0,Monday,13:15,14:15,59.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2139,79,X893,NaN,Walk,2021-03-12,24.0,Friday,16:00,17:00,60.0,2.0
2140,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,17:00,18:30,90.0,-1.0
2141,79,O733,NaN,Dinner,2021-03-12,24.0,Friday,18:30,19:00,30.0,2.0
2142,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,19:00,21:30,150.0,-1.0


In [24]:
# Add StartDateTime and EndDataTime columns
df['StartDateTime'] = df['StartDate'].dt.strftime("%Y-%m-%d") + ' ' + df['StartTime']
df['EndDateTime'] = df['StartDate'].dt.strftime("%Y-%m-%d") + ' ' + df['EndTime']
df['StartDateTime'] = pd.to_datetime(df['StartDateTime'], errors='coerce')
df['EndDateTime'] = pd.to_datetime(df['EndDateTime'], errors='coerce')

print("Added StartDateTime and EndDataTime columns")
print("NaN values in StartDateTime:", any(df['StartDateTime'] != df['StartDateTime']))
print("NaN values in EndDateTime:", any(df['EndDateTime'] != df['EndDateTime']))

Added StartDateTime and EndDataTime columns
NaN values in StartDateTime: True
NaN values in EndDateTime: True


In [25]:
df

,CSVNumber,PrimaryActivityCode,SecondaryActivityCode,DescriptiveLabel,StartDate,UniBathWeekNo,DayOfWeek,StartTime,EndTime,DurationMins,EnjoymentScore,StartDateTime,EndDateTime
2,0,X893,UD415,Exercising while listening music,2021-03-08,24.0,Monday,07:30,08:15,45.0,2.0,2021-03-08 07:30:00,2021-03-08 08:15:00
4,0,CW982,NaN,ADS Lab 3,2021-03-08,24.0,Monday,08:45,11:15,150.0,1.0,2021-03-08 08:45:00,2021-03-08 11:15:00
5,0,H179,NaN,Room and desk cleaning,2021-03-08,24.0,Monday,11:15,11:45,29.0,-1.0,2021-03-08 11:15:00,2021-03-08 11:45:00
6,0,CW982,NaN,Preparing for ENTP presentation,2021-03-08,24.0,Monday,11:45,12:45,60.0,1.0,2021-03-08 11:45:00,2021-03-08 12:45:00
8,0,EL642,UD415,Online ENTP lecture,2021-03-08,24.0,Monday,13:15,14:15,59.0,0.0,2021-03-08 13:15:00,2021-03-08 14:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,79,X893,NaN,Walk,2021-03-12,24.0,Friday,16:00,17:00,60.0,2.0,2021-03-12 16:00:00,2021-03-12 17:00:00
31,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,17:00,18:30,90.0,-1.0,2021-03-12 17:00:00,2021-03-12 18:30:00
32,79,O733,NaN,Dinner,2021-03-12,24.0,Friday,18:30,19:00,30.0,2.0,2021-03-12 18:30:00,2021-03-12 19:00:00
33,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,19:00,21:30,150.0,-1.0,2021-03-12 19:00:00,2021-03-12 21:30:00


In [26]:
mask = (df['StartDateTime'] == df['StartDateTime']) | (df['EndDateTime'] == df['EndDateTime'])
df = df[mask]
print("Dropped rows with invalid datetime values")

Dropped rows with invalid datetime values


In [27]:
df.reset_index(drop=True)

,CSVNumber,PrimaryActivityCode,SecondaryActivityCode,DescriptiveLabel,StartDate,UniBathWeekNo,DayOfWeek,StartTime,EndTime,DurationMins,EnjoymentScore,StartDateTime,EndDateTime
0,0,X893,UD415,Exercising while listening music,2021-03-08,24.0,Monday,07:30,08:15,45.0,2.0,2021-03-08 07:30:00,2021-03-08 08:15:00
1,0,CW982,NaN,ADS Lab 3,2021-03-08,24.0,Monday,08:45,11:15,150.0,1.0,2021-03-08 08:45:00,2021-03-08 11:15:00
2,0,H179,NaN,Room and desk cleaning,2021-03-08,24.0,Monday,11:15,11:45,29.0,-1.0,2021-03-08 11:15:00,2021-03-08 11:45:00
3,0,CW982,NaN,Preparing for ENTP presentation,2021-03-08,24.0,Monday,11:45,12:45,60.0,1.0,2021-03-08 11:45:00,2021-03-08 12:45:00
4,0,EL642,UD415,Online ENTP lecture,2021-03-08,24.0,Monday,13:15,14:15,59.0,0.0,2021-03-08 13:15:00,2021-03-08 14:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2130,79,X893,NaN,Walk,2021-03-12,24.0,Friday,16:00,17:00,60.0,2.0,2021-03-12 16:00:00,2021-03-12 17:00:00
2131,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,17:00,18:30,90.0,-1.0,2021-03-12 17:00:00,2021-03-12 18:30:00
2132,79,O733,NaN,Dinner,2021-03-12,24.0,Friday,18:30,19:00,30.0,2.0,2021-03-12 18:30:00,2021-03-12 19:00:00
2133,79,EL642,UD415,NaN,2021-03-12,24.0,Friday,19:00,21:30,150.0,-1.0,2021-03-12 19:00:00,2021-03-12 21:30:00


In [28]:
# check EnjoymentScore
df.EnjoymentScore.unique()

array([ 2.,  1., -1.,  0., -2., nan])

In [29]:
print(df['StartDateTime'].min())
print(df['EndDateTime'].max())

2021-02-23 08:30:00
2021-03-12 23:45:00


In [30]:
# validate DurationMins
# DurationMins should be <= 1440
mask = (df['DurationMins'] <= 1440) # date that this coursework was set and due
df = df.loc[mask]
print("Dropped rows where an activity duration is longer than a day")

Dropped rows where an activity duration is longer than a day


In [31]:
df.to_csv('processed_data.csv', index=False)

### Questions to ask on the dataset:

The questions are formulated as follows:
1. Are there any activities that the students have spent more time on daytime compared to the people in the additional dataset? 
2. How much time have the students slept on average, and is the difference in sleep time significant when compared to the people in the additional dataset?
3. Do the EnjoymentScore and Sleep (code: S801) have a positive correlation? In other words, does the amount of sleep affect the enjoyment scores of the activities?

For the additional dataset, the European Time Use dataset by (Eurostat, 2010) and organised by (Dai, 2019) will be used. This dataset includes all age group.

Reference: 

Eurostat 2010, Time spent, participation time and participation rate in the main activity by sex and day of the week, viewed 28 March 2021, <https://ec.europa.eu/eurostat/data/database?node_code=tus_00week>.

Dai, S 2019, European Time Use, viewed 28 March 2021, <https://www.kaggle.com/yuchendai/european-time-use>.

#### Importing additional datasets for comparison

In [39]:
European_TimeUse = pd.read_csv("European_TimeUse.csv")

# add seconds to time strings to convert into timedelta
European_TimeUse.iloc[:,2:] = European_TimeUse.iloc[:,2:] + ':00'
European_TimeUse.iloc[:,2:] = European_TimeUse.iloc[:,2:].apply(pd.to_timedelta, errors='coerce')
European_TimeUse = European_TimeUse.groupby('GEO/ACL00').sum().drop('SEX', axis=1) / 2 # aggregate males and females
European_TimeUse = European_TimeUse / 10**9 / 60 # nanoseconds to minutes

European_TimeUse_mean = European_TimeUse.mean()
European_TimeUse_mean.sort_values(ascending=False)

Total                                                                              1440.000000
Personal care                                                                       654.607143
Sleep                                                                               508.821429
Leisure, social and associative life                                                295.464286
Household and family care                                                           202.892857
Employment, related activities and travel as part of/during main and second job     196.892857
Main and second job and related travel                                              194.142857
TV and video                                                                        127.321429
Eating                                                                               98.214286
Travel except travel related to jobs                                                 73.678571
Other and/or unspecified personal care            

#### Question 1: Are there any activities that the students have spent more time on daytime compared to the people in the additional dataset?

Activity codes:
- Eating / Drinking (Code: ED145) 
- Education / Lectures (Code: EL642) 
- Exercise (Code: X893) 
- Housework (Code: H179) 
- Leisure (Code: L418) 
- Reading (Code: R523) 
- Sleep (Code: S801) 
- Travelling (Code: T695)
- Using Devices (Code: UD415) 
- Paid Work (Code: PW101)
- Coursework (Code: CW982) 
- Other (Code: O733)

For each individual csv, the same primary activities have to be merged
Then, this question can be answered by:
1. Match the PrimaryActivityCode on the wrangled dataset df and the activities on the European_TimeUse dataset.
2. Plot a histogram for visual comparison.
3. Analyse the difference.

In [43]:
# df.groupby(['PrimaryActivityCode', 'CSVNumber', 'DayOfWeek']).sum().groupby(level=0).mean()
PrimaryActivityCode_DurationMins_mean = df.groupby(['PrimaryActivityCode']).mean()['DurationMins']
PrimaryActivityCode_DurationMins_mean

PrimaryActivityCode
CW982    134.871720
ED145     48.543147
EL642    113.634518
H179      41.890625
L418      93.992806
O733      40.164474
PW101    157.454545
R523      85.731707
S801     380.313564
T695      47.325581
UD415     72.692857
X893      69.474138
Name: DurationMins, dtype: float64

In [107]:
index = ['ED145', 'EL642', 'X893', 'H179', 'L418', 'R523', 'S801', 'T695', 'UD415', 'PW101', 'CW982']
index_name = ['Eating / Drinking', 'Education / Lectures', 'Exercise', 'Housework',
             'Leisure', 'Reading', 'Sleep', 'Travelling', 'Using Devices',
             'Paid Work', 'Coursework']

# q1_original and q1_additional for q1 histogram plot
q1_original = PrimaryActivityCode_DurationMins_mean.loc[index]  # wrangled dataset mean

# extract and match the activity categories to q1_original by appending to q1_additional array 
# from European TimeUse dataset mean
# and convert to numpy array
q1_additional = []
q1_additional.append(European_TimeUse_mean['Eating'])
q1_additional.append(European_TimeUse_mean['Study'] + European_TimeUse_mean['School and university except homework'])
q1_additional.append(European_TimeUse_mean['Walking and hiking'] + European_TimeUse_mean['Sports and outdoor activities except walking and hiking'])
q1_additional.append(European_TimeUse_mean['Household and family care'])
q1_additional.append(European_TimeUse_mean['Leisure, social and associative life'])
q1_additional.append(European_TimeUse_mean['Reading, except books'] + European_TimeUse_mean['Reading books'])
q1_additional.append(European_TimeUse_mean['Sleep'])
q1_additional.append(European_TimeUse_mean['Travel except travel related to jobs'] +
                     European_TimeUse_mean['Travel related to leisure, social and associative life'] +
                     European_TimeUse_mean['Travel to/from work'] +
                     European_TimeUse_mean['Travel related to shopping and services'])
q1_additional.append(European_TimeUse_mean['TV and video'])
q1_additional.append(European_TimeUse_mean['Employment, related activities and travel as part of/during main and second job'] +
                     European_TimeUse_mean['Main and second job and related travel'])
q1_additional.append(European_TimeUse_mean['Homework'])

q1_original = q1_original.rename("Wrangled")
print(q1_original)
q1_additional = pd.Series(q1_additional, index=index)
q1_additional = q1_additional.rename("Additional")
print(q1_additional)

# q1_original = q1_original.to_numpy()
# q1_additional = np.array(q1_additional)

print(q1_original.shape, q1_additional.shape)

q1 = pd.concat([q1_original, q1_additional], axis=1)

PrimaryActivityCode
ED145     48.543147
EL642    113.634518
X893      69.474138
H179      41.890625
L418      93.992806
R523      85.731707
S801     380.313564
T695      47.325581
UD415     72.692857
PW101    157.454545
CW982    134.871720
Name: Wrangled, dtype: float64
ED145     98.214286
EL642     17.892857
X893      26.250000
H179     202.892857
L418     295.464286
R523      27.035714
S801     508.821429
T695     133.535714
UD415    127.321429
PW101    391.035714
CW982      4.750000
Name: Additional, dtype: float64
(11,) (11,)


In [85]:
q1

,Wrangled,Additional
PrimaryActivityCode,,
ED145,48.543147,98.214286
EL642,113.634518,17.892857
X893,69.474138,26.250000
H179,41.890625,202.892857
L418,93.992806,295.464286
R523,85.731707,27.035714
S801,380.313564,508.821429
T695,47.325581,133.535714
UD415,72.692857,127.321429


In [121]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Bath Students', x=index_name, y=q1.Wrangled),
    go.Bar(name='Average', x=index_name, y=q1.Additional)
])
# Change the bar mode
fig.update_layout(
    barmode='group', 
    colors='T10',
    font_family="Calibri",
    xaxis_title="Primary activity codes",
    yaxis_title="Time spent (minutes)",
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(yanchor="top", y=1.0, xanchor="right", x=0.99),
    title={'text': "Average Time Spent in One Day", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'middle'}
    )

fig.update_xaxes(showgrid=True, linecolor='black')
fig.update_yaxes(showgrid=True, linecolor='black', gridcolor='gray', gridwidth=1)

fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'colors'

Did you mean "coloraxis"?

    Valid properties:
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        angularaxis
            :class:`plotly.graph_objects.layout.AngularAxis`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regardless 

#### Question 2

In [36]:
# average sleep time from european dataset
SleepTime = European_TimeUse['Sleep']

SleepTime = SleepTime + ':00' # to match time format for to_timedelta function
SleepTime = pd.to_timedelta(SleepTime)

Sleep_seconds_mean = SleepTime.mean().seconds
Sleep_hours_mean = Sleep_seconds_mean//3600
Sleep_minutes_mean = (Sleep_seconds_mean//60)%60

print("European mean sleep time: {} hours {} minutes".format(Sleep_hours_mean, Sleep_minutes_mean))

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [ ]:
SleepTime